<a href="https://colab.research.google.com/github/Thayyebsalim/music_analyisis/blob/master/Music_Sentimental_analysis_NLP_by_Tone_and_Lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro

The idea of this code is to find the mood of the song in any way possible helping us to segregate songs better.

1.   First part is gonna be scraping the lyrics from any website or API 
2.   Finding the vibe of the song with sentimental analysis
3.   Fetch and mp3 version and check the mood with transfer learning hopefully or something similar



In [1]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.3 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelea

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import re
import time
import string
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("vader_lexicon")
nltk.download("brown")
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

We need to scrape the song lyrics from websites or API to use here

I am using selenium here and the speed is not the best at the moment so I might switch to scrapy or use an API

In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


def get_track_link(track_name): # Gets track name from user and converts it so that we can use it in a link
  #track_name = input("Enter a track name: ")
  track_name_link = track_name.rstrip().replace(" ","%20").lower()
  return track_name_link


def scrape_tracklist(track_link): # Scrapes the page for list of tracks with similar names or such 
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get("https://www.musixmatch.com/search/" + track_link + "/tracks")
  element = driver.find_element_by_class_name("box-content")
  link_list = [i.get_attribute('href') for i in element.find_elements_by_tag_name('a')]
  driver.close()
  lyric_links=[i for i in link_list if re.search("/lyrics/",i)] #could do it in one line but for the sake of clarity, I'm gonna do two
  lyric_links=[i for i in lyric_links if not re.search("/add",i)]
  return lyric_links


def scrape_lyrics(lyric_links_list,choice): # Scrapes the lyrics element
  driver = webdriver.Chrome(chrome_options=chrome_options)
  driver.get(lyric_links_list[choice])
  lyric_elem=driver.find_elements_by_class_name('mxm-lyrics__content')
  lyric_elem= [i.find_element_by_tag_name('span') for i in lyric_elem] 
  lyric_elem= [i.get_attribute('innerHTML') for i in lyric_elem]
  driver.close()
  return lyric_elem


def select_track(artist_track_list): # Lets the user select a track
  sel=0
  for track in artist_track_list:
    print(f"{track[0]+1} {track[1]} - {track[2]}")
  sel=int(input('Enter the number of your desired track: '))-1
  return sel


def get_lyric_element(track_link,choice=0): # Cleans up the lyric element and returns a cleaned list(array) with just lyrics inside
  lyric_links_list = scrape_tracklist(track_link)
  print(lyric_links_list)
  artist_track_list = [[lyric_links_list.index(x), x.split("/")[4],x.split('/')[5]] for x in lyric_links_list]
  print(artist_track_list)
  if len(artist_track_list) > 1 and choice != "default":
    choice = select_track(artist_track_list)
  elif len(artist_track_list) == 0 :
    return "Sorry! no such song"
  else:
    choice=0
  lyric_element = scrape_lyrics(lyric_links_list,choice)
  return " ".join(lyric_element)



In [ ]:
track_name= "grandson war"
track_link= get_track_link(track_name) #uncomment to activate function
track_link

In [ ]:
lyric_element = get_lyric_element(track_link,"default")
lyric_element

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/Grandson/War']
[[0, 'Grandson', 'War']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


"Until the philosophy, we chose one race superior\nAnd another\nInferior He's finally\nAnd permanently\nDiscredited and abandoned\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nEverywhere is war\nIt's a war\nOh, everywhere is war\nIt's a war\n\nAnd until there's no longer\nFirst and second class citizens of any nation\nIt's on\nAnd until the color of a man's skin serves no more significance\nThan the color of his eyes\nWe say war\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\n\nAnd it's war in the east\nWar in the west\nWar up north\nWar down south\nEverywhere is\nOh, everywhere is war\n\nIt's a war\nOh, everywhere is war\nIt's a war\nOh, everywhere is war\n\nLa, la, la, la, la, la, la\nLa, la, la, la, la, la, la\n(Everywhere is, everywhere is, everywhere is)\n(Everywhere is, everywhere is, everywhere is)\n\nEverywhere is war\nEverywhere is war"

The first part is done, web scraper should work fine. I did not implement error cases for now becuase I want to focus on the next two parts of the project


In [4]:
def filter_raw_lyrics(raw_lyr): #The scraped element has blanks and \n's lets filter those 
  return [i for i in raw_lyr.split("\n") if i != '']

#joiner
def convert_arr_to_str(arr):
  return " ".join(arr)

In [ ]:
lyrics_as_array= filter_raw_lyrics(lyric_element)
lyrics=convert_arr_to_str(lyrics_as_array) #returns a single string as lyrics
lyrics,lyrics_as_array

("Until the philosophy, we chose one race superior And another Inferior He's finally And permanently Discredited and abandoned And it's war in the east War in the west War up north War down south And it's war in the east War in the west War up north War down south Everywhere is war It's a war Oh, everywhere is war It's a war And until there's no longer First and second class citizens of any nation It's on And until the color of a man's skin serves no more significance Than the color of his eyes We say war And it's war in the east War in the west War up north War down south And it's war in the east War in the west War up north War down south Everywhere is Oh, everywhere is war It's a war Oh, everywhere is war It's a war Oh, everywhere is war La, la, la, la, la, la, la La, la, la, la, la, la, la (Everywhere is, everywhere is, everywhere is) (Everywhere is, everywhere is, everywhere is) Everywhere is war Everywhere is war",
 ['Until the philosophy, we chose one race superior',
  'And anot

Well lets have some fun with the lyrics

In [5]:
def swfilter(word): #function we use in filter()
  return word not in stopwords.words()

def remove_stopword(lyr_arr):
  return [" ".join(filter(swfilter,sentence.split())) for sentence in lyr_arr]

In [6]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

def map_contraction(word): #function we use in map()
  word=word.lower()
  if word in contractions:
    return contractions[word]
  else: 
    return word

def remove_contraction(lyr_arr):
  return [" ".join(map(map_contraction, sentence.split())) for sentence in lyr_arr]

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()

def map_lemmatize(word):
  return wordnet_lemmatizer.lemmatize(word, pos="v")

def lemmatize_words(lyr_arr):
  return [" ".join(map(map_lemmatize, sentence.split())) for sentence in lyr_arr]

In [ ]:
lyrics_as_array_without_sw = remove_stopword(lyrics_as_array)
lyrics_without_sw = convert_arr_to_str(lyrics_as_array_without_sw)
lyrics_as_array_without_sw,lyrics_without_sw

(['Until philosophy, chose race superior',
  'And another',
  "Inferior He's finally",
  'And permanently',
  'Discredited abandoned',
  'And east',
  'War west',
  'War north',
  'War south',
  'And east',
  'War west',
  'War north',
  'War south',
  'Everywhere',
  "It's",
  'Oh, everywhere',
  "It's",
  "And there's longer",
  'First second class citizens nation',
  "It's",
  "And color man's skin serves significance",
  'Than color eyes',
  'We say',
  'And east',
  'War west',
  'War north',
  'War south',
  'And east',
  'War west',
  'War north',
  'War south',
  'Everywhere',
  'Oh, everywhere',
  "It's",
  'Oh, everywhere',
  "It's",
  'Oh, everywhere',
  'La, la, la, la, la, la,',
  'La, la, la, la, la, la,',
  '(Everywhere is, everywhere is, everywhere is)',
  '(Everywhere is, everywhere is, everywhere is)',
  'Everywhere',
  'Everywhere'],
 "Until philosophy, chose race superior And another Inferior He's finally And permanently Discredited abandoned And east War west War n

In [ ]:
lyrics_as_array_without_contraction = remove_contraction(lyrics_as_array)
lyrics_without_contraction = convert_arr_to_str(lyrics_as_array_without_contraction)
lyrics_as_array_without_contraction,lyrics_without_contraction

(['until the philosophy, we chose one race superior',
  'and another',
  'inferior he has / he is finally',
  'and permanently',
  'discredited and abandoned',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'everywhere is war',
  'it has / it is a war',
  'oh, everywhere is war',
  'it has / it is a war',
  'and until there has / there is no longer',
  'first and second class citizens of any nation',
  'it has / it is on',
  "and until the color of a man's skin serves no more significance",
  'than the color of his eyes',
  'we say war',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'and it has / it is war in the east',
  'war in the west',
  'war up north',
  'war down south',
  'everywhere is',
  'oh, everywhere is war',
  'it has / it is a war',
  'oh, everywhere is war',
  'it 

In [ ]:
lyrics_as_array_with_lemmatized_words=lemmatize_words(lyrics_as_array)
lyrics_with_lemmatized_words=convert_arr_to_str(lyrics_as_array_with_lemmatized_words)
lyrics_as_array_with_lemmatized_words,lyrics_with_lemmatized_words

(['Until the philosophy, we choose one race superior',
  'And another',
  "Inferior He's finally",
  'And permanently',
  'Discredited and abandon',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  'Everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  "It's a war",
  "And until there's no longer",
  'First and second class citizens of any nation',
  "It's on",
  "And until the color of a man's skin serve no more significance",
  'Than the color of his eye',
  'We say war',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  "And it's war in the east",
  'War in the west',
  'War up north',
  'War down south',
  'Everywhere be',
  'Oh, everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  "It's a war",
  'Oh, everywhere be war',
  'La, la, la, la, la, la, la',
  'La, la, la, la, la, la, la',
  '(E


Next we have to tackle the sentimantal analysis part. We aim to try bag of words then use NLTK on every line might be slow but I like to give prominence to accuracy

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()
def get_vader_score(lyr):
  # Polarity score returns dictionary
  sentiment = sia.polarity_scores(lyr)
  return sentiment['compound']
  """
  print(ss)
  print(ss['compound'])
  ss.pop("compound")
  print("ss=",ss)
  x=sorted(ss.values(), reverse=True)[0]
  for i in ss.keys():
    if ss[i]==x:
      if "i" == "neg":
        ss[i]=-(ss[i])
      print(ss[i])
  """  

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
get_vader_score(lyrics)

-0.999

In [9]:
def get_textblob_score(lyr):
  blob=TextBlob(lyr)
  return blob.polarity

In [ ]:
get_textblob_score(lyrics)

0.008641975308641962

In [10]:
!pip install watson_developer_cloud
import sys
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions, CategoriesOptions
natural_language_understanding = NaturalLanguageUnderstandingV1(                                         
    version='2018-11-16',
    iam_apikey="hzkkbjivKNHfoi3cFlMBOPG6-YHLc-7CAQKwXn8Rt7JT", 
    url="https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/512aa79c-a8ba-43fa-b9d5-a4382099b06f" # paste the url here
    )
def get_watson_score(input_text): 
  try:
    response = natural_language_understanding.analyze (
    text = input_text,
    features = Features(sentiment=SentimentOptions())).get_result()
    res = response.get('sentiment').get('document').get('score')
    return res
  except Exception:
    exc_type, value, traceback = sys.exc_info()
    print("error",input_text,exc_type.__name__)
    return 0

     |████████████████████████████████| 256kB 3.2MB/s 
     |████████████████████████████████| 204kB 10.5MB/s 
  Created wheel for watson-developer-cloud: filename=watson_developer_cloud-2.10.1-cp36-none-any.whl size=252814 sha256=068c287337a9d5d8a495355fba8ebdaa3c9dfdde209ae380897ec1849afac3be
  Stored in directory: /root/.cache/pip/wheels/ea/a7/19/fac0a408a586265eb374005308a7553d4494ab41b2fd88f5bc
Successfully built watson-developer-cloud


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: watson-developer-cloud moved to ibm-watson. To get updates, use the new package.
  


In [ ]:
get_watson_score(lyrics)

-0.777161

In [ ]:
sentiment_score_fns=[get_textblob_score,get_vader_score]

In [11]:
# Returns all transformed arrays and further maps the arrays 

def get_lyric_arrays(lyr_arr):
  return [lyr_arr,
          remove_stopword(lyr_arr),
          remove_contraction(lyr_arr),
          lemmatize_words(lyr_arr),
          remove_contraction(remove_stopword(lyr_arr)),
          lemmatize_words(remove_stopword(lyr_arr)),
          remove_contraction(lemmatize_words(lyr_arr)),
          lemmatize_words(remove_contraction(remove_stopword(lyr_arr)))]

def get_lyrics(arr):
  return list(map(convert_arr_to_str,arr))


In [ ]:

list_of_converted_lyr_arr=get_lyric_arrays(lyrics_as_array) #returns a tuple with 3 items lyrics, lyrics without stop wordsm 
list_of_converted_lyr=get_lyrics(list_of_converted_lyr_arr)

In [12]:
import pandas as pd 
import numpy as np

In [ ]:
df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-CO-LM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df["Textblob"]=list(map(get_textblob_score,list_of_converted_lyr))
df["Vader"]=list(map(get_vader_score,list_of_converted_lyr))
df["Watson"]=list(map(get_watson_score,list_of_converted_lyr))

In [ ]:
textblob_per_line=[]
for trans in list_of_converted_lyr_arr:
  textblob_per_line.append(list(map(get_textblob_score,trans)))

df["Textblob per line"]=list(map(np.mean,textblob_per_line))
#textblob_per_line

In [ ]:
vader_per_line=[]
for trans in list_of_converted_lyr_arr:
  vader_per_line.append(list(map(get_vader_score,trans)))

df["Vader per line"]=list(map(np.mean,vader_per_line))
#vader_per_line

In [13]:
def watson_min_words_arr(arr): #watson has minimum requirement to perform sentiment analysis
  x="" 
  watson_arr=[]
  for line in arr:
    if len(line.split())+len(x.split())<4:
      x=x+line+" "
      continue
    if x!= "" : line=x+line
    x=""
    watson_arr.append(line)
  return watson_arr 



In [ ]:
converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))
converted_watson_arr

[['Until the philosophy, we chose one race superior',
  "And another Inferior He's finally",
  'And permanently Discredited and abandoned',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  "Everywhere is war It's a war",
  'Oh, everywhere is war',
  "It's a war And until there's no longer",
  'First and second class citizens of any nation',
  "It's on And until the color of a man's skin serves no more significance",
  'Than the color of his eyes',
  "We say war And it's war in the east",
  'War in the west',
  'War up north War down south',
  "And it's war in the east",
  'War in the west',
  'War up north War down south',
  'Everywhere is Oh, everywhere is war',
  "It's a war Oh, everywhere is war",
  "It's a war Oh, everywhere is war",
  'La, la, la, la, la, la, la',
  'La, la, la, la, la, la, la',
  '(Everywhere is, everywhere is, everywhere is)',
  '(Everywhere i

In [ ]:
converted_watson_arr=list(map(watson_min_words_arr,list_of_converted_lyr_arr))

watson_per_line=[]
for trans in converted_watson_arr:
  watson_per_line.append(list(map(get_watson_score,trans)))

df["Watson per line"]=list(map(np.mean,watson_per_line))
#watson_per_line

error La, la, la, la, la, la, la
error La, la, la, la, la, la, la
error La, la, la, la, la, la,
error La, la, la, la, la, la,
error la, la, la, la, la, la, la
error la, la, la, la, la, la, la
error La, la, la, la, la, la, la
error La, la, la, la, la, la, la
error la, la, la, la, la, la,
error La, la, la, la, la, la,
error La, la, la, la, la, la,
error la, la, la, la, la, la, la
error la, la, la, la, la, la, la
error la, la, la, la, la, la,


In [ ]:
print(len(watson_per_line),len(converted_watson_arr))
for i in range(8):
  print(len(watson_per_line[i]),len(converted_watson_arr[i]))

8 8
29 29
22 22
33 33
29 29
24 24
22 22
33 33
24 24


In [14]:
def mean_without_zero(arr):
  arr = [x for x in arr if x != 0]
  return np.mean(arr)

In [ ]:
df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))

In [ ]:
df

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.149415,0.046778,0.233889,0.9599,0.095550,0.281029,-0.778259,-0.118538,-0.269406
SW,0.196875,0.053000,0.331250,0.9894,0.100832,0.315100,-0.386954,-0.020349,-0.063953
CO,0.149415,0.046778,0.233889,0.9953,0.098372,0.289329,-0.773304,-0.111188,-0.252699
LM,0.147222,0.054778,0.248990,0.9659,0.099942,0.293947,-0.649890,-0.030253,-0.072030
SW-CO,0.196875,0.053000,0.331250,0.9953,0.103654,0.323919,-0.492278,-0.018453,-0.059314
SW-LM,0.186842,0.061000,0.338889,0.9905,0.107898,0.337181,-0.289106,0.010841,0.029814
CO-LM,0.147222,0.054778,0.248990,0.9960,0.102764,0.302247,-0.622290,-0.025782,-0.061385
SW-CO-LM,0.186842,0.061000,0.338889,0.9957,0.110720,0.346000,-0.356586,0.010238,0.028794


In [15]:
def get_df(lyr_arr):
  trans_lyr_arr=get_lyric_arrays(lyr_arr)
  trans_lyr=get_lyrics(trans_lyr_arr)
  df=pd.DataFrame(index=["Lyrics","SW","CO","LM","SW-CO","SW-LM","CO-LM","SW-CO-LM"],columns=["Textblob","Textblob per line","Textblob nonzero","Vader","Vader per line","Vader nonzero","Watson","Watson per line","Watson nonzero"])
  df["Textblob"]=list(map(get_textblob_score,trans_lyr))
  df["Vader"]=list(map(get_vader_score,trans_lyr))
  df["Watson"]=list(map(get_watson_score,trans_lyr))

  textblob_per_line=[]
  for trans in trans_lyr_arr:
    textblob_per_line.append(list(map(get_textblob_score,trans)))

  df["Textblob per line"]=list(map(np.mean,textblob_per_line))
  #textblob_per_line

  vader_per_line=[]
  for trans in trans_lyr_arr:
    vader_per_line.append(list(map(get_vader_score,trans)))

  df["Vader per line"]=list(map(np.mean,vader_per_line))
  #vader_per_line

  converted_watson_arr=list(map(watson_min_words_arr,trans_lyr_arr))

  watson_per_line=[]
  for trans in converted_watson_arr:
    print(trans)
    watson_per_line.append(list(map(get_watson_score,trans)))
    
  df["Watson per line"]=list(map(np.mean,watson_per_line))
  #watson_per_line

  df["Textblob nonzero"]=list(map(mean_without_zero,textblob_per_line))
  df["Vader nonzero"]=list(map(mean_without_zero,vader_per_line))
  df["Watson nonzero"]=list(map(mean_without_zero,watson_per_line))
  return df



In [102]:
songs=["jacob lee demons","june messed up","magic rude","youre somebody else flora cash","idfc blackbear"]
song_sentiment=[-1,-1,1,-1,-1]
song_arr=list(map(get_track_link,songs))
song_arr

['jacob%20lee%20demons',
 'june%20messed%20up',
 'magic%20rude',
 'youre%20somebody%20else%20flora%20cash',
 'idfc%20blackbear']

In [103]:
elt_arr=[]
for i in song_arr:
  print(i)
  elt_arr.append(get_lyric_element(i,"default"))
elt_arr

jacob%20lee%20demons


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/Jacob-Lee/Demons']
[[0, 'Jacob-Lee', 'Demons']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


june%20messed%20up


...

idfc%20blackbear


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: use options instead of chrome_options
  


['https://www.musixmatch.com/lyrics/Black-Bear-3/Idfc', 'https://www.musixmatch.com/lyrics/Blackbear/Idfc', 'https://www.musixmatch.com/lyrics/blackbear-11/Idfc-Mixed']
[[0, 'Black-Bear-3', 'Idfc'], [1, 'Blackbear', 'Idfc'], [2, 'blackbear-11', 'Idfc-Mixed']]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: use options instead of chrome_options


["My heart is weak\nTear it down piece by piece\nLeave me to think Deep in my structure, I think I still love her\nBut I need some sleep\nYou've taken my breath away, now I want to breathe\nCause I cannot see, what you can see, so easily\n\nI thought my demons, were almost defeated\nBut you took their side, &amp; you pulled them to freedom\nI kept your secrets &amp; I thought that you would do the same\n\nLeave me in peace\nCaught in my memories, lost underneath\nDeep in my structure, I feel a rupture, from where she should be\nYou've taken my breath from me, now I want to breathe\nCause I cannot see, what you can see, so easily\n\nI thought my demons, were almost defeated\nBut you took their side, &amp; you pulled them to freedom\nThey know my secrets &amp; won't let me go, won't let me go\nI thought my demons, were almost defeated\nBut you took their side, &amp; you pulled them to freedom\nI kept your secrets &amp; I thought that you would do the same\n\nI thought my demons, were alm

In [104]:
getdffrom=list(map(filter_raw_lyrics,elt_arr))
getdffrom

[['My heart is weak',
  'Tear it down piece by piece',
  'Leave me to think Deep in my structure, I think I still love her',
  'But I need some sleep',
  "You've taken my breath away, now I want to breathe",
  'Cause I cannot see, what you can see, so easily',
  'I thought my demons, were almost defeated',
  'But you took their side, &amp; you pulled them to freedom',
  'I kept your secrets &amp; I thought that you would do the same',
  'Leave me in peace',
  'Caught in my memories, lost underneath',
  'Deep in my structure, I feel a rupture, from where she should be',
  "You've taken my breath from me, now I want to breathe",
  'Cause I cannot see, what you can see, so easily',
  'I thought my demons, were almost defeated',
  'But you took their side, &amp; you pulled them to freedom',
  "They know my secrets &amp; won't let me go, won't let me go",
  'I thought my demons, were almost defeated',
  'But you took their side, &amp; you pulled them to freedom',
  'I kept your secrets &amp

In [105]:
accuracy=list(map(get_df,getdffrom))

['My heart is weak', 'Tear it down piece by piece', 'Leave me to think Deep in my structure, I think I still love her', 'But I need some sleep', "You've taken my breath away, now I want to breathe", 'Cause I cannot see, what you can see, so easily', 'I thought my demons, were almost defeated', 'But you took their side, &amp; you pulled them to freedom', 'I kept your secrets &amp; I thought that you would do the same', 'Leave me in peace', 'Caught in my memories, lost underneath', 'Deep in my structure, I feel a rupture, from where she should be', "You've taken my breath from me, now I want to breathe", 'Cause I cannot see, what you can see, so easily', 'I thought my demons, were almost defeated', 'But you took their side, &amp; you pulled them to freedom', "They know my secrets &amp; won't let me go, won't let me go", 'I thought my demons, were almost defeated', 'But you took their side, &amp; you pulled them to freedom', 'I kept your secrets &amp; I thought that you would do the same'

In [106]:
for i in accuracy:
  display(i)

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.083611,0.022543,0.117222,0.9633,0.098012,0.141572,-0.371796,-0.108447,-0.108447
SW,0.165278,0.028526,0.185417,0.8748,0.035312,0.057381,-0.335312,-0.173610,-0.245097
CO,0.083611,0.022543,0.117222,0.9813,0.125204,0.180850,-0.436700,-0.110671,-0.110671
LM,0.083611,0.022543,0.117222,0.9638,0.098542,0.142339,0.338803,-0.087847,-0.087847
SW-CO,0.165278,0.028526,0.185417,0.9244,0.062504,0.101569,-0.356591,-0.173156,-0.244456
SW-LM,0.165278,0.028526,0.185417,0.8777,0.035842,0.058244,0.336744,-0.155735,-0.219861
CO-LM,0.083611,0.022543,0.117222,0.9816,0.125735,0.181617,-0.294294,-0.089940,-0.089940
SW-CO-LM,0.165278,0.028526,0.185417,0.9265,0.063035,0.102431,0.323240,-0.153256,-0.216362


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.010779,0.015184,0.024884,0.9222,0.019734,0.029108,-0.462724,-0.131651,-0.165330
SW,0.022917,0.023093,0.041288,-0.9740,0.021612,0.032695,-0.651018,-0.286566,-0.345565
CO,0.010779,0.015184,0.024884,0.9339,0.022315,0.032915,-0.358686,-0.129806,-0.163012
LM,-0.017444,0.038806,0.067341,0.8243,0.009876,0.014568,-0.572606,-0.191016,-0.234429
SW-CO,0.022917,0.023093,0.041288,-0.9822,0.024193,0.036600,-0.656212,-0.286129,-0.345038
SW-LM,-0.010227,0.043326,0.082460,-0.9799,0.011754,0.017782,-0.708590,-0.295793,-0.356691
CO-LM,-0.017444,0.038806,0.067341,0.8243,0.012458,0.018375,-0.475647,-0.189171,-0.232164
SW-CO-LM,0.004792,0.057733,0.094618,-0.9874,0.013617,0.020600,-0.713785,-0.295724,-0.356608


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,-0.184364,-0.080954,-0.202385,-0.9976,-0.111284,-0.172683,-0.714440,-0.126322,-0.138646
SW,-0.184364,-0.080954,-0.202385,0.6010,-0.004749,-0.007369,-0.652718,-0.009885,-0.012709
CO,-0.184364,-0.080954,-0.202385,-0.9982,-0.111284,-0.172683,-0.712121,-0.107508,-0.117997
LM,-0.189490,-0.080105,-0.200263,-0.9975,-0.109804,-0.170386,-0.710571,-0.126741,-0.139106
SW-CO,-0.184364,-0.080954,-0.202385,-0.3866,-0.004749,-0.007369,-0.637354,0.040404,0.046921
SW-LM,-0.189490,-0.080105,-0.200263,0.7004,-0.000596,-0.000924,-0.655593,-0.014746,-0.018959
CO-LM,-0.189490,-0.080105,-0.200263,-0.9981,-0.109804,-0.170386,-0.708099,-0.107927,-0.118456
SW-CO-LM,-0.189490,-0.080105,-0.200263,0.5192,-0.000596,-0.000924,-0.638911,0.037736,0.043823


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,0.051042,0.008654,0.064286,0.9802,0.071538,0.143077,-0.406107,0.019814,0.028556
SW,0.028205,0.007051,0.052381,0.9802,0.071538,0.143077,0.270524,-0.093218,-0.147186
CO,0.022917,0.008654,0.064286,0.9896,0.074492,0.148985,-0.448666,0.009187,0.013241
LM,0.051042,0.008654,0.064286,0.9837,0.080362,0.167152,-0.409649,0.003645,0.005252
SW-CO,0.028205,0.007051,0.052381,0.9896,0.074492,0.148985,-0.379344,-0.054936,-0.078822
SW-LM,0.028205,0.007051,0.052381,0.9837,0.080362,0.167152,0.277929,-0.059449,-0.093867
CO-LM,0.022917,0.008654,0.064286,0.9916,0.083315,0.173296,-0.440917,-0.006546,-0.009434
SW-CO-LM,0.028205,0.007051,0.052381,0.9916,0.083315,0.173296,-0.344133,-0.032764,-0.047009


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero
Lyrics,-0.220833,-0.135897,-0.294444,-0.9831,-0.090518,-0.141208,-0.972300,-0.428174,-0.477109
SW,-0.265000,-0.135897,-0.294444,0.9806,0.107374,0.167504,-0.956526,-0.526227,-0.621905
CO,0.154167,0.094872,0.205556,-0.9854,-0.106046,-0.165432,-0.980172,-0.492500,-0.548786
LM,-0.120833,-0.074359,-0.161111,0.9552,0.048513,0.075680,-0.983283,-0.523772,-0.619003
SW-CO,-0.265000,-0.135897,-0.294444,0.9882,0.123097,0.192032,-0.980808,-0.383192,-0.487699
SW-LM,-0.145000,-0.074359,-0.161111,-0.7136,-0.032949,-0.051400,-0.988043,-0.631569,-0.746400
CO-LM,0.129167,0.079487,0.172222,0.9862,0.051436,0.080240,-0.985010,-0.486889,-0.575414
SW-CO-LM,-0.145000,-0.074359,-0.161111,-0.9027,-0.036867,-0.057512,-0.994903,-0.444161,-0.565295


In [107]:
newset=["","","","",""]
for i in range(len(accuracy)):
  if song_sentiment[i]==1:
    newset[i]=(accuracy[i]+1)/2
  else:
    newset[i]=(accuracy[i]-1)/-2
  newset[i]=round(newset[i]*100,1)

In [108]:
for i in newset:
  i["mean_row"]=i.mean(axis=1)
  i["max_row"]=i.max(axis=1)

In [109]:
for i in newset:
  display(i)

,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero,mean_row,max_row
Lyrics,45.8,48.9,44.1,1.8,45.1,42.9,68.6,55.4,55.4,45.333333,68.6
SW,41.7,48.6,40.7,6.3,48.2,47.1,66.8,58.7,62.3,46.711111,66.8
CO,45.8,48.9,44.1,0.9,43.7,41.0,71.8,55.5,55.5,45.244444,71.8
LM,45.8,48.9,44.1,1.8,45.1,42.9,33.1,54.4,54.4,41.166667,54.4
SW-CO,41.7,48.6,40.7,3.8,46.9,44.9,67.8,58.7,62.2,46.144444,67.8
SW-LM,41.7,48.6,40.7,6.1,48.2,47.1,33.2,57.8,61.0,42.711111,61.0
CO-LM,45.8,48.9,44.1,0.9,43.7,40.9,64.7,54.5,54.5,44.222222,64.7
SW-CO-LM,41.7,48.6,40.7,3.7,46.8,44.9,33.8,57.7,60.8,42.077778,60.8


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero,mean_row,max_row
Lyrics,49.5,49.2,48.8,3.9,49.0,48.5,73.1,56.6,58.3,48.544444,73.1
SW,48.9,48.8,47.9,98.7,48.9,48.4,82.6,64.3,67.3,61.755556,98.7
CO,49.5,49.2,48.8,3.3,48.9,48.4,67.9,56.5,58.2,47.855556,67.9
LM,50.9,48.1,46.6,8.8,49.5,49.3,78.6,59.6,61.7,50.344444,78.6
SW-CO,48.9,48.8,47.9,99.1,48.8,48.2,82.8,64.3,67.3,61.788889,99.1
SW-LM,50.5,47.8,45.9,99.0,49.4,49.1,85.4,64.8,67.8,62.188889,99.0
CO-LM,50.9,48.1,46.6,8.8,49.4,49.1,73.8,59.5,61.6,49.755556,73.8
SW-CO-LM,49.8,47.1,45.3,99.4,49.3,49.0,85.7,64.8,67.8,62.022222,99.4


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero,mean_row,max_row
Lyrics,40.8,46.0,39.9,0.1,44.4,41.4,14.3,43.7,43.1,34.855556,46.0
SW,40.8,46.0,39.9,80.0,49.8,49.6,17.4,49.5,49.4,46.933333,80.0
CO,40.8,46.0,39.9,0.1,44.4,41.4,14.4,44.6,44.1,35.077778,46.0
LM,40.5,46.0,40.0,0.1,44.5,41.5,14.5,43.7,43.0,34.866667,46.0
SW-CO,40.8,46.0,39.9,30.7,49.8,49.6,18.1,52.0,52.3,42.133333,52.3
SW-LM,40.5,46.0,40.0,85.0,50.0,50.0,17.2,49.3,49.1,47.455556,85.0
CO-LM,40.5,46.0,40.0,0.1,44.5,41.5,14.6,44.6,44.1,35.100000,46.0
SW-CO-LM,40.5,46.0,40.0,76.0,50.0,50.0,18.1,51.9,52.2,47.188889,76.0


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero,mean_row,max_row
Lyrics,47.4,49.6,46.8,1.0,46.4,42.8,70.3,49.0,48.6,44.655556,70.3
SW,48.6,49.6,47.4,1.0,46.4,42.8,36.5,54.7,57.4,42.711111,57.4
CO,48.9,49.6,46.8,0.5,46.3,42.6,72.4,49.5,49.3,45.100000,72.4
LM,47.4,49.6,46.8,0.8,46.0,41.6,70.5,49.8,49.7,44.688889,70.5
SW-CO,48.6,49.6,47.4,0.5,46.3,42.6,69.0,52.7,53.9,45.622222,69.0
SW-LM,48.6,49.6,47.4,0.8,46.0,41.6,36.1,53.0,54.7,41.977778,54.7
CO-LM,48.9,49.6,46.8,0.4,45.8,41.3,72.0,50.3,50.5,45.066667,72.0
SW-CO-LM,48.6,49.6,47.4,0.4,45.8,41.3,67.2,51.6,52.4,44.922222,67.2


,Textblob,Textblob per line,Textblob nonzero,Vader,Vader per line,Vader nonzero,Watson,Watson per line,Watson nonzero,mean_row,max_row
Lyrics,39.0,43.2,35.3,0.8,45.5,42.9,1.4,28.6,26.1,29.200000,45.5
SW,36.8,43.2,35.3,99.0,55.4,58.4,2.2,23.7,18.9,41.433333,99.0
CO,57.7,54.7,60.3,0.7,44.7,41.7,1.0,25.4,22.6,34.311111,60.3
LM,44.0,46.3,41.9,97.8,52.4,53.8,0.8,23.8,19.0,42.200000,97.8
SW-CO,36.8,43.2,35.3,99.4,56.2,59.6,1.0,30.8,25.6,43.100000,99.4
SW-LM,42.8,46.3,41.9,14.3,48.4,47.4,0.6,18.4,12.7,30.311111,48.4
CO-LM,56.5,54.0,58.6,99.3,52.6,54.0,0.7,25.7,21.2,46.955556,99.3
SW-CO-LM,42.8,46.3,41.9,4.9,48.2,47.1,0.3,27.8,21.7,31.222222,48.2


Kinda done with this part. Looks okay for now. Now we got to see if we can get an mp3 version of the song and see if the song sounds sad or happy. 